In [2]:
import gpytorch
import torch
import numpy as np
import metrics

In [3]:
train_x = torch.Tensor([0, 0])
train_y = torch.Tensor([0.1, 0.1])

In [4]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [5]:
adam = torch.optim.Adam(model.parameters(), 0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
for i in range(50):
    output = model(train_x)
    loss = -mll(output, train_x)
    loss.backward() 
    adam.step()


In [6]:
output = model(train_x)
loss = -mll(output, train_x)
metrics.calculate_laplace(model, -loss, with_prior=True, param_punish_term=-1.0)


(tensor(-1.2098, dtype=torch.float64, grad_fn=<AddBackward0>),
 {'neg MLL': tensor(-1.7903, grad_fn=<NegBackward0>),
  'punish term': tensor(-3.0001, dtype=torch.float64),
  'punish without replacement': tensor(inf, dtype=torch.float64),
  'num_replaced': tensor(3),
  'parameter list': ['likelihood.noise_covar.raw_noise',
   'covar_module.raw_outputscale',
   'covar_module.base_kernel.raw_lengthscale'],
  'parameter values': tensor([[-5.9927],
          [-5.9931],
          [ 0.0000]]),
  'corrected Hessian': tensor([[ 4.6427e+01, -6.5769e-16,  0.0000e+00],
          [ 1.1284e-15,  4.6427e+01,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  4.6427e+01]], dtype=torch.float64),
  'diag(constructed eigvals)': tensor([46.4268, 46.4268, 46.4268], dtype=torch.float64),
  'original symmetrized Hessian': tensor([[ 0.0639, -0.0540,  0.0000],
          [-0.0540,  0.0559,  0.0000],
          [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64),
  'prior mean': tensor([[-3.5164],
          [-1.6

In [7]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    output = model(train_x)
    observed_pred = likelihood(output)
    

/home/besginow/anaconda3/envs/sage/lib/python3.10/site-packages/gpytorch/models/exact_gp.py:274: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [8]:
observed_pred.covariance_matrix

tensor([[0.0034, 0.0009],
        [0.0009, 0.0034]])

In [9]:
observed_pred.loc

tensor([0.0658, 0.0658])

In [10]:
from metrics import calculate_mc_STAN
# Perform MCMC
MCMC_approx, MC_log = calculate_mc_STAN(
    model, likelihood, 1000)
MC_logs = dict()
print(MCMC_approx)
print(MC_log)

RuntimeError: asyncio.run() cannot be called from a running event loop